## Deriving New Variables

In [20]:
# Task: Import necessary libraries
import pandas as pd

# Task: Import ords_prods_merge.csv from the "Prepared Data" folder
ords_prods_merge_path = "/Users/ging/Documents/Data Analytics Course/04_2024_Instacart Basket Analysis/02 Data/Prepared Data/ords_prods_merge.csv"
ords_prods_merge = pd.read_csv(ords_prods_merge_path)

In [21]:
# Task: Work with a subset of the dataframe - the first one million rows
print("\nTask: Work with a subset of the dataframe - the first one million rows")
df = ords_prods_merge[:1000000]



Task: Work with a subset of the dataframe - the first one million rows


In [24]:
import pandas as pd

# Define the price_label function
def price_label(row):
    if row['prices'] <= 5:
        return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid-range product'
    elif row['prices'] > 15:
        return 'High-range product'
    else:
        return 'Not enough data'

# Read the dataframe (assuming you have already loaded it)
# df = ...

# Apply the price_label function to create a new column 'price_range' using .loc to avoid SettingWithCopyWarning
df.loc[:, 'price_range'] = df.apply(price_label, axis=1)

# Print the value counts of the new column
print(df['price_range'].value_counts())


price_range
Mid-range product     674474
Low-range product     312624
High-range product     12902
Name: count, dtype: int64


In [26]:
## Printing frequency of orders day of the week
print(df['order_dow'].value_counts())


order_dow
0    190938
1    174445
6    139984
2    131601
5    129053
3    118708
4    115271
Name: count, dtype: int64


In [29]:
# Increase IOPub data rate limit
%config InlineBackend.figure_format = 'retina'
%config InlineBackend.print_figure_kwargs = {'facecolor': 'white'}
%config InlineBackend.rc = {'figure.dpi': 300, 'savefig.dpi': 300}

# Initialize empty list
result = []

# Loop through every value in "order_dow" column
for value in ords_prods_merge["order_dow"]:
    if value == 0:
        result.append("Busiest day")
    elif value == 4:
        result.append("Least busy")
    else:
        result.append("Regularly busy")

# Add "busiest_day" column to the dataframe
ords_prods_merge.loc[:, "busiest_day"] = result

# Print frequency of new column
print(ords_prods_merge["busiest_day"].value_counts())


busiest_day
Regularly busy    22437387
Busiest day        6209632
Least busy         3787193
Name: count, dtype: int64


In [30]:
# Print frequency of new column "busiest_day"
print("Frequency of busiest_day column:")
print(ords_prods_merge["busiest_day"].value_counts())

# Print frequency of "order_dow" column for cross-checking
print("\nFrequency of order_dow column:")
print(ords_prods_merge["order_dow"].value_counts())


Frequency of busiest_day column:
busiest_day
Regularly busy    22437387
Busiest day        6209632
Least busy         3787193
Name: count, dtype: int64

Frequency of order_dow column:
order_dow
0    6209632
1    5665830
6    4500246
2    4217766
5    4209449
3    3844096
4    3787193
Name: count, dtype: int64


In [31]:
# Calculate frequency of each day
day_frequency = ords_prods_merge['order_dow'].value_counts()

# Find the two busiest days and two slowest days
busiest_days = day_frequency.nlargest(2).index.tolist()
slowest_days = day_frequency.nsmallest(2).index.tolist()

# Create a new column with updated labels
ords_prods_merge['updated_busiest_day'] = ords_prods_merge['order_dow'].apply(
    lambda x: 'Busiest days' if x in busiest_days else ('Slowest days' if x in slowest_days else 'Regular days')
)

# Print frequency of the new column
print("Frequency of updated_busiest_day column:")
print(ords_prods_merge['updated_busiest_day'].value_counts())


Frequency of updated_busiest_day column:
updated_busiest_day
Regular days    12927461
Busiest days    11875462
Slowest days     7631289
Name: count, dtype: int64


In [33]:
# Print frequency of updated_busiest_day column
print("Frequency of updated_busiest_day column:")
print(ords_prods_merge['updated_busiest_day'].value_counts())

# Print frequency of order_dow column
print("\nFrequency of order_dow column:")
print(ords_prods_merge['order_dow'].value_counts())


Frequency of updated_busiest_day column:
updated_busiest_day
Regular days    12927461
Busiest days    11875462
Slowest days     7631289
Name: count, dtype: int64

Frequency of order_dow column:
order_dow
0    6209632
1    5665830
6    4500246
2    4217766
5    4209449
3    3844096
4    3787193
Name: count, dtype: int64


## 

Results
1) The updated_busiest_day column has three categories: Regular days, Busiest days, and Slowest days.
2) In the order_dow column, the highest count is for day 0 (Saturday), followed by day 1 (Sunday), which aligns with our expectations for the busiest days.
3) The lowest counts are for days 4 (Wednesday) and 3 (Thursday), confirming our expectation for the slowest days.

Observations:
1) The frequency counts for the busiest and slowest days in the updated_busiest_day column align with the frequency counts for the corresponding days in the order_dow column.
2) This indicates that the labels in the updated_busiest_day column accurately represent the two busiest and two slowest days of the week, as per the order_dow column.
3) The labels "Regular days" likely correspond to the remaining weekdays, which don't fall into either the busiest or slowest categories.

In [35]:
# Define a function to categorize hours into busiest, average, and fewest orders
def categorize_hour(hour_count):
    if hour_count > busiest_threshold:
        return "Most orders"
    elif hour_count < fewest_threshold:
        return "Fewest orders"
    else:
        return "Average orders"

# Calculate the threshold values for busiest and fewest orders
hourly_order_counts = ords_prods_merge.groupby('order_hour_of_day').size()
busiest_threshold = hourly_order_counts.max() * 0.7  # 70% of the maximum order count
fewest_threshold = hourly_order_counts.min() * 1.3    # 130% of the minimum order count

# Apply the categorization function to create the new column
ords_prods_merge['busiest_period_of_day'] = ords_prods_merge['order_hour_of_day'].apply(categorize_hour)

# Print the frequency for the new column
print(ords_prods_merge['busiest_period_of_day'].value_counts())


busiest_period_of_day
Fewest orders    32434212
Name: count, dtype: int64


In [36]:
import pandas as pd

# Define the file path for the pickle file
file_path = "/Users/ging/Documents/Data Analytics Course/04_2024_Instacart Basket Analysis/02 Data/Prepared Data/Excercise_4_7_Jha_New_Variables.pkl"

# Export the dataframe as a pickle file
ords_prods_merge.to_pickle(file_path)
